In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" 


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove-cache/glove-cache/glove_monant_5000vocab_300d.npy
/kaggle/input/glove-cache/glove-cache/glove_med_5000vocab_300d.npy
/kaggle/input/glove-cache/glove-cache/desktop.ini
/kaggle/input/glove-cache/glove-cache/glove_pubhealth_5000vocab_300d.npy
/kaggle/input/glove-6b-300d/glove.6B.300d.txt
/kaggle/input/final-data-for-training/Final data for training/Monant_claims.csv
/kaggle/input/final-data-for-training/Final data for training/desktop.ini
/kaggle/input/final-data-for-training/Final data for training/PUBHEALTH/test.tsv
/kaggle/input/final-data-for-training/Final data for training/PUBHEALTH/dev.tsv
/kaggle/input/final-data-for-training/Final data for training/PUBHEALTH/train.tsv
/kaggle/input/final-data-for-training/Final data for training/PUBHEALTH/desktop.ini
/kaggle/input/final-data-for-training/Final data for training/Med-MMHL/dev.csv
/kaggle/input/final-data-for-training/Final data for training/Med-MMHL/desktop.ini
/kaggle/input/final-data-for-training/Final data fo

In [2]:
# Install required packages for Excel output
!pip install -q openpyxl

# -*- coding: utf-8 -*-
"""BERT.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1Q0YHA6RUjwOClr1AKzK3nUcPlVusNKGa
"""


'BERT.ipynb\n\nAutomatically generated by Colab.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/1Q0YHA6RUjwOClr1AKzK3nUcPlVusNKGa\n'

In [3]:
# Check if GPU is Available
import torch

if torch.cuda.is_available():
    print("GPU is available.")
    device = torch.device("cuda")
else:
    print("GPU is NOT available.")
    print("Please go to Runtime > Change runtime type > Select GPU")
    device = torch.device("cpu")

GPU is available.


In [4]:
!pip install -q transformers datasets accelerate evaluate optuna

# ===== Imports ===== #
# Import all the libraries

import os
import random
import numpy as np
import pandas as pd
import torch
import json
import openpyxl
import sys
from pathlib import Path
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
from evaluate import load as load_metric
from time import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, precision_score
from sklearn.metrics import recall_score, f1_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

E0000 00:00:1752556722.241814      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752556722.342387      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# ===== Seed Everything (for full reproducibility) ===== #
import random
import numpy as np
import torch
import torch.backends.cudnn as cudnn

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False  # This makes sure cuDNN doesn’t auto-tune

set_seed(42)


In [6]:
# Remove old outputs directory (optional but recommended for clean experiments)
import shutil

shutil.rmtree("/kaggle/working/MSc_Claim_Experiment/outputs", ignore_errors=True)


In [7]:

# ===== Configurations ===== #

# model_name = "bert-base-uncased"  # Use the standard model √
# model_name = "distilbert-base-uncased"   √
# model_name = "roberta-base"  √
# model_name = "albert-base-v2"  √
model_name = "xlnet-base-cased" 
# model_name = "google/electra-base-discriminator" 

# lr = 2e-5  # Learning rate (changeable)


timestamp = datetime.now().isoformat()

# Generate a unique run ID with model name and timestamp
# run_id = f"{model_name.replace('/', '_')}_lr{str(lr).replace('.', '')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"


run_id = f"{model_name.replace('/', '_')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# ===== Environment-Aware Path Setup ===== #
if "google.colab" in sys.modules:
    # Running in Google Colab
    project_root = Path("/content/drive/MyDrive/MSc_Claim_Experiment")
elif Path("/kaggle/working").exists():
    # Running in Kaggle
    project_root = Path("/kaggle/working/MSc_Claim_Experiment")
else:
    # Fallback for local development
    project_root = Path("./MSc_Claim_Experiment")

# Define the output directory for the current run
# output_dir = project_root / "outputs" / "results" / run_id
output_dir = project_root / "outputs" / "results" / model_name.replace("/", "_") / run_id
os.makedirs(output_dir, exist_ok=True)  # Create directory if it does not exist

In [8]:
# ===== Load and Preprocess Data ===== #

# Select dataset name to control which source is loaded
# dataset_name = "monant" 
# dataset_name = "pubhealth" 
dataset_name = "med" 

if dataset_name == "monant":
    # Load Monant dataset and standardize labels
    # df = pd.read_csv("/kaggle/input/final-data-for-training/Monant_claims.csv", encoding="ISO-8859-1", engine="python")
    df = pd.read_csv("/kaggle/input/final-data-for-training/Final data for training/Monant_claims.csv", encoding="ISO-8859-1", engine="python")

    df['rating'] = df['rating'].str.lower().str.strip()  # Make ratings lowercase and remove spaces

     # Map 4-class labels to binary classification: 1 for true/mostly-true, 0 for false/mostly-false
    mapping = {
        'true': 1,
        'mostly-true': 1,
        'false': 0,
        'mostly-false': 0
}  
    
    # Apply label mapping
    df['label_binary'] = df['rating'].map(mapping)
    df = df[df['label_binary'].isin([0, 1])] # Keep only rows that are 0 or 1 (drop unknowns)

    # Split into train, validation, test
    train_val, test = train_test_split(df, test_size=0.15, stratify=df['label_binary'], random_state=42)
    train, val      = train_test_split(train_val, test_size=0.1765, stratify=train_val['label_binary'], random_state=42)

 # Ensure 'statement' is the standard text column for all datasets
    for df_ in [train, val, test]:
        df_['statement'] = df_['statement']  # Redundant if column already exists; safe fallback

elif dataset_name == "pubhealth":
    # Load PubHealth dataset (already split into train/dev/test)
    train = pd.read_csv("/kaggle/input/final-data-for-training/Final data for training/PUBHEALTH/train.tsv", sep="\t")
    val   = pd.read_csv("/kaggle/input/final-data-for-training/Final data for training/PUBHEALTH/dev.tsv", sep="\t")
    test  = pd.read_csv("/kaggle/input/final-data-for-training/Final data for training/PUBHEALTH/test.tsv", sep="\t")
   
    # Map labels to binary classification and align column names
    mapping = {'true': 1, 'mostly true': 1, 'false': 0, 'mostly false': 0}
    
    for df_ in [train, val, test]:
        # Standardize and map labels
        df_['label'] = df_['label'].astype(str).str.lower().str.strip()
        df_['label_binary'] = df_['label'].map(mapping)

        # Align main text field to common column name 'statement'
        df_['statement'] = df_['claim'] 

    # Keep only rows with valid binary labels
    train, val, test = [df[df['label_binary'].isin([0, 1])] for df in [train, val, test]]

elif dataset_name == "med":
    train = pd.read_csv("/kaggle/input/final-data-for-training/Final data for training/Med-MMHL/train.csv")
    val   = pd.read_csv("/kaggle/input/final-data-for-training/Final data for training/Med-MMHL/dev.csv")
    test  = pd.read_csv("/kaggle/input/final-data-for-training/Final data for training/Med-MMHL/test.csv")
   
      # Align field names for consistency across datasets
    for df_ in [train, val, test]:
        df_['label_binary'] = df_['det_fake_label']  # Already 0 or 1
        df_['statement'] = df_['content']            # Standardize text column
        df_.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')  # Drop extra index if exists

    # Optional: Ensure only binary labels 0 or 1 remain (in case of data corruption)
    train, val, test = [df[df['label_binary'].isin([0, 1])] for df in [train, val, test]]

In [9]:
# ===== Compute Class Weights =====
# Handle class imbalance by adjusting loss contribution of each class. Give rare class more weight
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train['label_binary']),
    y=train['label_binary']
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to('cuda' if torch.cuda.is_available() else 'cpu')

# Save class weights to JSON file for reproducibility
with open(output_dir / "class_weights.json", "w") as f:
    json.dump(class_weights.cpu().numpy().tolist(), f, indent=2)

# Show class distribution
print("Train:", train['label_binary'].value_counts(normalize=True))
print("Val:", val['label_binary'].value_counts(normalize=True))
print("Test:", test['label_binary'].value_counts(normalize=True))


Train: label_binary
0    0.70142
1    0.29858
Name: proportion, dtype: float64
Val: label_binary
0    0.701356
1    0.298644
Name: proportion, dtype: float64
Test: label_binary
0    0.701399
1    0.298601
Name: proportion, dtype: float64


In [10]:
# ===== Column Cleanup Before HuggingFace Dataset Conversion ===== #
columns_to_keep = ['statement', 'label_binary']
cleaned_train = train[columns_to_keep].copy()
cleaned_val   = val[columns_to_keep].copy()
cleaned_test  = test[columns_to_keep].copy()

# Combine for tokenization
dataset = Dataset.from_pandas(
    pd.concat([cleaned_train, cleaned_val, cleaned_test]).reset_index(drop=True)
)

In [11]:

# ===== Ensure Labels Are Integers (Fix for loss mismatch) ===== #
# Prevents loss function errors by ensuring labels are int (not string or float)
train = train.copy()
val = val.copy()
test = test.copy()

train["label_binary"] = train["label_binary"].astype(int)
val["label_binary"] = val["label_binary"].astype(int)
test["label_binary"] = test["label_binary"].astype(int)

In [12]:

# ===== Tokenizer ===== #
tokenizer = AutoTokenizer.from_pretrained(model_name)  # Load tokenizer

def tokenize(batch):
    # Try statement first, fall back to content
    if 'statement' in batch:
        texts = batch['statement']
    elif 'content' in batch:
        texts = batch['content']
    else:
        raise ValueError("No valid text field found in batch.")
    
    return tokenizer(texts, padding=True, truncation=True)  # Turn text into tokens

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

In [13]:
# ===== Columns to Remove During Cleanup ===== #
columns_to_remove = [
    'id', 'name', 'description', 'category', 'rating',
    'claim_id', 'source_url', 'queries', 'created_at', 'updated_at',
    'Unnamed: 0', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
    'statement', 'claim','content', 'date_published', 'explanation',
    'main_text', 'fact_checkers', 'sources', 'subjects'
]

In [14]:
# Remove unnecessary columns dynamically
def safe_remove_columns(dataset, columns):
    existing = [col for col in columns if col in dataset.column_names]
    return dataset.remove_columns(existing)

In [15]:
# ===== Unified Tokenize + Clean Function ===== #
def tokenize_and_clean(df):
    """
    Tokenize and clean a DataFrame for Transformer input.
    - Ensures labels are renamed to 'labels' (required by Trainer).
    - Removes unnecessary columns, including misleading 'label' if it's text.
    """

    # ===== Step 0: Convert to HuggingFace Dataset ===== #
    ds = Dataset.from_pandas(df, preserve_index=False)  

    # ===== Step 1: Identify and rename the correct label column ===== #
    if "label_binary" in ds.column_names:
        ds = ds.rename_column("label_binary", "labels")
    elif "det_fake_label" in ds.column_names:
        ds = ds.rename_column("det_fake_label", "labels")
    elif "label" in ds.column_names and isinstance(ds["label"][0], int):
        ds = ds.rename_column("label", "labels")
    else:
        raise ValueError("No valid label column found. Expecting 'label_binary' or integer 'label'.")

    # ===== Step 1.5: Ensure Labels Are Integer Type ===== #
    ds = ds.map(lambda x: {"labels": int(x["labels"])})


    # ===== Step 2: Remove 'label' column if it’s a string (textual label, not numeric) ===== #
    if "label" in ds.column_names and "labels" in ds.column_names:
        if isinstance(ds["label"][0], str):
            ds = ds.remove_columns("label")

    # ===== Step 3: Apply Tokenizer ===== #
    ds = ds.map(tokenize, batched=True)

    # Step 4: Keep only model-relevant fields
    model_inputs = {"input_ids", "attention_mask", "token_type_ids", "labels"}
    to_remove = [col for col in ds.column_names if col not in model_inputs]
    ds = ds.remove_columns(to_remove)

    # debug check
    print("Final columns after cleaning:", ds.column_names)
    return ds

In [16]:
# ===== Apply Processing Pipeline to Each Split ===== #
train_ds = tokenize_and_clean(train)
val_ds   = tokenize_and_clean(val)
test_ds  = tokenize_and_clean(test)

print("Final dataset columns:", train_ds.column_names) # Only 'input_ids', 'attention_mask', 'labels'

# Inspect first few samples
for i in range(3):
    print(f"\nSample {i}:")
    print(train_ds[i])

Map:   0%|          | 0/41282 [00:00<?, ? examples/s]

Map:   0%|          | 0/41282 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Final columns after cleaning: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']


Map:   0%|          | 0/5900 [00:00<?, ? examples/s]

Map:   0%|          | 0/5900 [00:00<?, ? examples/s]

Final columns after cleaning: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']


Map:   0%|          | 0/11795 [00:00<?, ? examples/s]

Map:   0%|          | 0/11795 [00:00<?, ? examples/s]

Final columns after cleaning: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']
Final dataset columns: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']

Sample 0:
{'labels': 1, 'input_ids': [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2678, 2706, 190, 50, 2383, 1779, 1091, 20, 594, 316, 1987, 7672, 254, 14043, 9743, 31791, 23, 9, 4, 3], 'token_type_ids': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [17]:
# ===== Initialize Model ===== #
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
def compute_metrics(pred):

    y_true = pred.label_ids # The true labels of the test set
    y_pred = pred.predictions.argmax(-1) # The model's predicted class (the one with the highest probability)

    # Calculate evaluation metrics for class 0
    precision = precision_score(y_true, y_pred, pos_label=0, zero_division=0)
    recall = recall_score(y_true, y_pred, pos_label=0, zero_division=0)
    f1 = f1_score(y_true, y_pred, pos_label=0, zero_division=0)

    # Overall accuracy of the model
    acc = accuracy_score(y_true, y_pred)

    # Calculate Weighted metrics
    f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    precision_weighted = precision_score(y_true, y_pred, average="weighted", zero_division=0)
    recall_weighted = recall_score(y_true, y_pred, average="weighted", zero_division=0)

    # Calculate micro-averaged precision, recall, and F1 (global average across all classes)
    precision_micro = precision_score(y_true, y_pred, average="micro", zero_division=0)
    recall_micro = recall_score(y_true, y_pred, average="micro", zero_division=0)
    f1_micro = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # Macro metrics
    precision_macro = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
    # Return all metrics in a dictionary
    return {
        'eval_f1': f1_weighted,
        "accuracy": acc,
        "precision": precision,
        "recall": recall,

        "precision_micro": precision_micro,
        "recall_micro": recall_micro,
        "f1_micro": f1_micro,

        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,

        "f1_weighted": f1_weighted,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,

        "support_1": int((y_true == 1).sum()),  # Number of class 1 samples
        "support_0": int((y_true == 0).sum())  # Number of class 0 samples
    }

In [19]:
# ===== Define Training Arguments ===== #
# Key parameters for reproducibility and fine-tuning control
# This block defines how training should proceed, how models are saved, and evaluation behavior

training_args = TrainingArguments(
    output_dir=output_dir,  # Folder to store model checkpoints and logs
    # evaluation_strategy='epoch',  # Evaluate model after each epoch
    eval_strategy='epoch', 
    save_strategy='epoch', # Save model after each epoch
    save_total_limit=1, # Keep only the latest saved model to reduce disk usage
    # learning_rate=2e-5,  # Initial learning rate for optimizer
    # learning_rate=3e-5, # Alternative setting for grid search
    learning_rate=5e-5, # # Max learning rate
    lr_scheduler_type="cosine",
    warmup_ratio=0.1, 
    per_device_train_batch_size=8,  # Number of samples per GPU/CPU during training
    per_device_eval_batch_size=8,  # Number of samples per GPU/CPU during validation
    num_train_epochs=100, # Max number of training epochs (early stopping can interrupt before 100)
    weight_decay=0.01, # L2 regularization to reduce overfitting
    load_best_model_at_end=True,  # Restore best model at the end of training based on eval metric
    metric_for_best_model = 'eval_f1',
    greater_is_better = True,
    logging_dir=f"logs/{run_id}", # Directory to save training logs
    logging_strategy='epoch', # Log metrics at the end of each epoch
    seed=42, # Set random seed for reproducibility
    fp16=True,  # Use mixed precision for faster training on supported GPUs
    remove_unused_columns=False,  # Retain all columns to prevent loss of important features
    report_to='none', # Do not report metrics to third-party tools (e.g., WandB)
    disable_tqdm=True,  # Disable tqdm progress bar to reduce console clutter in Colab
    eval_accumulation_steps=32,  # Accumulate gradients during evaluation to save memory
    no_cuda=False  # Use GPU if available
)

In [20]:
# ===== Custom Logging Callback for Per-Epoch Metrics ===== #
# This callback logs evaluation metrics (loss, accuracy, precision, recall, F1) after every epoch
# It writes the results into a CSV file "epoch_log.csv" for visualization or debugging

from transformers import TrainerCallback
import csv

class MetricsLoggerCallback(TrainerCallback):
    def __init__(self, log_file=output_dir / "epoch_log.csv"):
        self.log_file = log_file
        with open(self.log_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                "epoch", "learning_rate",
                "eval_loss", "accuracy", "precision", "recall", "f1"
            ])

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None or "eval_loss" not in logs:
            return  # Skip non-eval logs

        current_lr = logs.get("learning_rate", None)

        with open(self.log_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                round(state.epoch, 2),
                current_lr,
                logs.get("eval_loss"),
                logs.get("eval_accuracy"),
                logs.get("eval_precision"),
                logs.get("eval_recall"),
                logs.get("eval_f1")
            ])

In [21]:

# ===== Data Collator for Padding =====
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:

# ===== Custom Trainer with Class Weights =====
#  Enables weighted loss to address label imbalance in binary classification
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Use weighted cross entropy loss
        #loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        #loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))
        
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        
        num_labels = model.module.config.num_labels if hasattr(model, "module") else model.config.num_labels

        loss = loss_fct(logits.view(-1, num_labels), labels.view(-1))

        # loss = loss_fct(logits.view(-1, model.module.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss


# Instantiate Trainer with early stopping and epoch logger
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5), MetricsLoggerCallback()],
    data_collator=data_collator  
)

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

In [23]:
# Check the structure of a problematic sample
sample = train_ds[0]
print(sample)

# Check if 'statement' still exists before accessing
if 'statement' in sample:
    print("Statement type:", type(sample['statement']))
    print("Statement value:", sample['statement'])
else:
    print("'statement' field has been removed after tokenization (expected).")

{'labels': 1, 'input_ids': [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2678, 2706, 190, 50, 2383, 1779, 1091, 20, 594, 316, 1987, 7672, 254, 14043, 9743, 31791, 23, 9, 4, 3], 'token_type_ids': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2], 'attention_mask': [0, 0, 0, 0, 0, 0, 0

In [24]:
# ===== Train Model with Timer ===== #
# Record how long training took
start_time = time()
trainer.train()
end_time = time()
train_time = round(end_time - start_time, 2)

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.3401, 'grad_norm': 0.10667973011732101, 'learning_rate': 4.998062766369624e-06, 'epoch': 1.0}
{'eval_f1': 0.9581661151569278, 'eval_loss': 0.5583905577659607, 'eval_accuracy': 0.9577966101694916, 'eval_precision': 0.9833457618692518, 'eval_recall': 0.9560173997100049, 'eval_precision_micro': 0.9577966101694916, 'eval_recall_micro': 0.9577966101694916, 'eval_f1_micro': 0.9577966101694916, 'eval_precision_macro': 0.943191261328872, 'eval_recall_macro': 0.9589962140434247, 'eval_f1_macro': 0.9505318208078093, 'eval_f1_weighted': 0.9581661151569278, 'eval_precision_weighted': 0.9593619551058115, 'eval_recall_weighted': 0.9577966101694916, 'eval_support_1': 1762, 'eval_support_0': 4138, 'eval_runtime': 59.1795, 'eval_samples_per_second': 99.697, 'eval_steps_per_second': 6.235, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.3845, 'grad_norm': 0.05664745345711708, 'learning_rate': 9.998062766369625e-06, 'epoch': 2.0}
{'eval_f1': 0.9658732009260265, 'eval_loss': 0.5646498203277588, 'eval_accuracy': 0.9659322033898305, 'eval_precision': 0.9733108920413561, 'eval_recall': 0.9782503624939585, 'eval_precision_micro': 0.9659322033898305, 'eval_recall_micro': 0.9659322033898305, 'eval_f1_micro': 0.9659322033898305, 'eval_precision_macro': 0.9608082317759912, 'eval_recall_macro': 0.9576268838576489, 'eval_f1_macro': 0.959197493592761, 'eval_f1_weighted': 0.9658732009260265, 'eval_precision_weighted': 0.9658432014015008, 'eval_recall_weighted': 0.9659322033898305, 'eval_support_1': 1762, 'eval_support_0': 4138, 'eval_runtime': 58.959, 'eval_samples_per_second': 100.069, 'eval_steps_per_second': 6.259, 'epoch': 2.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


...

{'loss': 0.1981, 'grad_norm': 0.31435084342956543, 'learning_rate': 3.999806276636963e-05, 'epoch': 8.0}
{'eval_f1': 0.9685847078709224, 'eval_loss': 0.4878431260585785, 'eval_accuracy': 0.9684745762711865, 'eval_precision': 0.982657547630679, 'eval_recall': 0.972208796520058, 'eval_precision_micro': 0.9684745762711865, 'eval_recall_micro': 0.9684745762711865, 'eval_f1_micro': 0.9684745762711865, 'eval_precision_macro': 0.9594904570933018, 'eval_recall_macro': 0.9659568386686556, 'eval_f1_macro': 0.962637601485181, 'eval_f1_weighted': 0.9685847078709224, 'eval_precision_weighted': 0.9688201193164898, 'eval_recall_weighted': 0.9684745762711865, 'eval_support_1': 1762, 'eval_support_0': 4138, 'eval_runtime': 59.019, 'eval_samples_per_second': 99.968, 'eval_steps_per_second': 6.252, 'epoch': 8.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.4655, 'grad_norm': 1611603.875, 'learning_rate': 4.499806276636963e-05, 'epoch': 9.0}
{'eval_f1': 0.13735600299076667, 'eval_loss': 0.746648907661438, 'eval_accuracy': 0.2986440677966102, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_precision_micro': 0.2986440677966102, 'eval_recall_micro': 0.2986440677966102, 'eval_f1_micro': 0.2986440677966102, 'eval_precision_macro': 0.1493220338983051, 'eval_recall_macro': 0.5, 'eval_f1_macro': 0.22996606630122682, 'eval_f1_weighted': 0.13735600299076667, 'eval_precision_weighted': 0.08918827923010629, 'eval_recall_weighted': 0.2986440677966102, 'eval_support_1': 1762, 'eval_support_0': 4138, 'eval_runtime': 58.8856, 'eval_samples_per_second': 100.194, 'eval_steps_per_second': 6.266, 'epoch': 9.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.7029, 'grad_norm': 1061809.125, 'learning_rate': 4.999806276636962e-05, 'epoch': 10.0}
{'eval_f1': 0.5782448391029342, 'eval_loss': 0.6940978169441223, 'eval_accuracy': 0.7013559322033899, 'eval_precision': 0.7013559322033899, 'eval_recall': 1.0, 'eval_precision_micro': 0.7013559322033899, 'eval_recall_micro': 0.7013559322033899, 'eval_f1_micro': 0.7013559322033899, 'eval_precision_macro': 0.35067796610169494, 'eval_recall_macro': 0.5, 'eval_f1_macro': 0.4122335126519227, 'eval_f1_weighted': 0.5782448391029342, 'eval_precision_weighted': 0.49190014363688594, 'eval_recall_weighted': 0.7013559322033899, 'eval_support_1': 1762, 'eval_support_0': 4138, 'eval_runtime': 58.9899, 'eval_samples_per_second': 100.017, 'eval_steps_per_second': 6.255, 'epoch': 10.0}
{'train_runtime': 17143.8443, 'train_samples_per_second': 240.798, 'train_steps_per_second': 15.055, 'train_loss': 0.3096932270415113, 'epoch': 10.0}


In [ ]:
!nvidia-smi

In [ ]:
!ls -lt /kaggle/working/MSc_Claim_Experiment/outputs/results/xlnet-base-cased/xlnet-base-cased_2025071*

In [29]:
#  Save training loss history (for reproducibility / visualization)
with open(output_dir / "train_loss_curve.json", "w") as f:
    json.dump(trainer.state.log_history, f, indent=2)

# ===== Save training time ===== #
with open(output_dir / "training_time_log.json", "w") as f:
    json.dump({"train_time_seconds": train_time}, f, indent=2)


In [26]:
print(f"Model loaded from: {trainer.model.base_model_prefix}, current LR: {trainer.optimizer.param_groups[0]['lr']}")

Model loaded from: transformer, current LR: 5e-05


In [27]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import torch
from tqdm.auto import tqdm # 确保 tqdm 在 Notebook 中显示
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import json

print("开始手动预测测试集...")

# 获取在训练结束时加载的最佳模型
# trainer.model 此时应该已经加载了最佳权重（因为 training_args 中设置了 load_best_model_at_end=True）
model_to_predict = trainer.model
model_to_predict.eval() # 将模型设置为评估模式
# 确保模型在正确的设备上（GPU 如果可用）
model_to_predict.to(device) # 'device' 变量在 Cell 3 中定义了

# 使用 DataCollatorWithPadding 确保测试数据批次正确填充
# return_tensors="pt" 确保返回 PyTorch 张量
data_collator_for_prediction = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
test_dataloader = DataLoader(
    test_ds,
    batch_size=trainer.args.per_device_eval_batch_size, # 使用训练参数中定义的评估批次大小
    collate_fn=data_collator_for_prediction,
    shuffle=False # 预测时不打乱数据顺序
)

all_logits = []
all_true_labels = []
# 从原始的 'test' DataFrame 中获取对应的 'statement'
# 注意：test_ds 是 tokenized 后的数据集，可能移除了原始列
# 我们需要从原始的 'test' DataFrame 中提取文本，并确保顺序一致
original_statements = test['statement'].tolist() # 'test' DataFrame 在 Cell 8 中定义

with torch.no_grad(): # 在预测过程中禁用梯度计算，节省内存和时间
    for batch_idx, batch in enumerate(tqdm(test_dataloader, desc="手动预测进度")):
        # 将输入数据移动到模型所在的设备
        inputs = {k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask', 'token_type_ids']}

        # 执行模型前向传播
        outputs = model_to_predict(**inputs)

        # 从模型输出中提取 logits，并移回 CPU
        logits = outputs.logits.cpu()
        all_logits.append(logits)

        # 收集真实标签 (如果批次中有标签)
        if 'labels' in batch:
            all_true_labels.extend(batch['labels'].cpu().numpy())

# 合并所有批次的 logits
final_logits = torch.cat(all_logits, dim=0).numpy()
# 根据 logits 确定预测标签
final_preds_labels = np.argmax(final_logits, axis=-1)

print("手动预测完成。")

# ===== 保存原始预测表 (prediction.csv) ===== #
# 确保 'test' DataFrame 和 test_ds 的样本顺序是匹配的
# 否则需要一个更复杂的映射机制
# 由于您的数据加载和分词流程通常会保持顺序，这里假设它们是匹配的。
results_df = pd.DataFrame({
    'statement': original_statements, # 使用从原始 DataFrame 提取的 statement
    'true_label': all_true_labels, # 使用手动收集的真实标签
    'pred_label': final_preds_labels,
    'logits': final_logits.tolist(), # 将 logits 转换为列表形式以便保存到 CSV
    'model_name': model_name,
    'run_id': run_id
})
results_df.to_csv(output_dir / "prediction.csv", index=False)
print(f"✅ prediction.csv 已包含所有列 (statement, true_label, pred_label, logits, model_name, run_id)，并保存至 {output_dir / 'prediction.csv'}")


# ===== 计算并保存评估指标 ===== #
# 使用手动预测的结果计算分类报告和混淆矩阵
conf_matrix = confusion_matrix(all_true_labels, final_preds_labels)
report = classification_report(all_true_labels, final_preds_labels, output_dict=True, zero_division=0)

with open(output_dir / "classification_report.json", "w") as f:
    json.dump(report, f, indent=2)
print(f"✅ classification_report.json 已保存至 {output_dir / 'classification_report.json'}")

with open(output_dir / "confusion_matrix.json", "w") as f:
    json.dump(conf_matrix.tolist(), f, indent=2)
print(f"✅ confusion_matrix.json 已保存至 {output_dir / 'confusion_matrix.json'}")

# 将 eval_metrics 也保存到 json 文件
# 确保这里的 metrics 结构与 compute_metrics 函数的输出保持一致
# 这里我们手动构造 preds_for_metrics 来兼容 compute_metrics 函数
class ManualPredictionOutput:
    def __init__(self, label_ids, predictions):
        self.label_ids = label_ids
        self.predictions = predictions
manual_preds_for_metrics = ManualPredictionOutput(np.array(all_true_labels), final_logits)
eval_metrics = compute_metrics(manual_preds_for_metrics)

with open(output_dir / "eval_metrics.json", "w") as f:
    json.dump(eval_metrics, f, indent=2)
print(f"✅ eval_metrics.json 已保存至 {output_dir / 'eval_metrics.json'}")

# ===== 保存训练时间和学习率等配置信息 (原 Cell 25, 33, 34) ===== #
# 这部分保持与原有 Notebook 中一致，只是确保它们在手动预测之后运行

# 保存训练损失历史
with open(output_dir / "train_loss_curve.json", "w") as f:
    json.dump(trainer.state.log_history, f, indent=2)
print(f"✅ train_loss_curve.json 已保存至 {output_dir / 'train_loss_curve.json'}")

# 保存训练时间
# train_time 变量应该在 trainer.train() 之后被设置
with open(output_dir / "training_time_log.json", "w") as f:
    json.dump({"train_time_seconds": train_time}, f, indent=2)
print(f"✅ training_time_log.json 已保存至 {output_dir / 'training_time_log.json'}")

# 估计学习率 & 早停 epoch
best_lr = trainer.optimizer.param_groups[0]['lr'] # 从 Trainer 获取最终学习率
# 这里我们直接使用 trainer.state.epoch 来获取早停时的 epoch，因为它更准确
epoch_stopped_at = trainer.state.epoch

print(f"Final learning rate (from best checkpoint): {best_lr}")
print(f"Early stopped at epoch: {int(epoch_stopped_at)}")

with open(output_dir / "best_lr.txt", "w") as f:
    f.write(str(best_lr))
print(f"✅ best_lr.txt 已保存至 {output_dir / 'best_lr.txt'}")

# 保存模型 & 训练配置快照
config_snapshot = {
    "model_name": model_name,
    "run_id": run_id,
    "learning_rate": training_args.learning_rate,
    "lr_scheduler_type": training_args.lr_scheduler_type,
    "warmup_ratio": training_args.warmup_ratio,
    "batch_size": training_args.per_device_train_batch_size,
    "num_epochs": training_args.num_train_epochs,
    "weight_decay": training_args.weight_decay,
    "early_stopping": True,
    "best_lr": best_lr,
    "epoch_stopped_at": int(epoch_stopped_at),
    "class_weights": class_weights.cpu().tolist(),
    "datetime": timestamp
}
with open(output_dir / "config.json", "w") as f:
    json.dump(config_snapshot, f, indent=2)
print(f"✅ config.json 已保存至 {output_dir / 'config.json'}")

# 写入可读的运行信息
with open(output_dir / "README.txt", "w") as f:
    f.write(f"Run ID: {run_id}\nModel: {model_name}\nCreated: {timestamp}\n")
    f.write(f"Best LR (from checkpoint): {best_lr}\nStopped at epoch: {int(epoch_stopped_at)}\n")
print(f"✅ README.txt 已保存至 {output_dir / 'README.txt'}")

# 保存 TrainingArguments Params (原 Cell 35 开头部分)
with open(output_dir / "training_args.json", "w") as f:
    json.dump(training_args.to_dict(), f, indent=2)
print(f"✅ training_args.json 已保存至 {output_dir / 'training_args.json'}")

print(f"🎉 所有结果已保存至总目录: {output_dir}")

开始手动预测测试集...


手动预测进度:   0%|          | 0/1475 [00:00<?, ?it/s]

手动预测完成。
✅ prediction.csv 已包含所有列 (statement, true_label, pred_label, logits, model_name, run_id)，并保存至 /kaggle/working/MSc_Claim_Experiment/outputs/results/xlnet-base-cased/xlnet-base-cased_20250715_051900/prediction.csv
✅ classification_report.json 已保存至 /kaggle/working/MSc_Claim_Experiment/outputs/results/xlnet-base-cased/xlnet-base-cased_20250715_051900/classification_report.json
✅ confusion_matrix.json 已保存至 /kaggle/working/MSc_Claim_Experiment/outputs/results/xlnet-base-cased/xlnet-base-cased_20250715_051900/confusion_matrix.json
✅ eval_metrics.json 已保存至 /kaggle/working/MSc_Claim_Experiment/outputs/results/xlnet-base-cased/xlnet-base-cased_20250715_051900/eval_metrics.json
✅ train_loss_curve.json 已保存至 /kaggle/working/MSc_Claim_Experiment/outputs/results/xlnet-base-cased/xlnet-base-cased_20250715_051900/train_loss_curve.json
✅ training_time_log.json 已保存至 /kaggle/working/MSc_Claim_Experiment/outputs/results/xlnet-base-cased/xlnet-base-cased_20250715_051900/training_time_log.json
Final l

In [30]:
# ===== Append Main Metrics to Excel Summary Table ===== #
# 创建/更新一个 Excel 总结表格，用于最终模型比较
summary_path = project_root / "outputs" / "results" / "final_model_comparison.xlsx"

# 构建一行 summary 数据，使用手动预测的结果
summary_row = {
    "model_name": model_name,
    "run_id": run_id,
    "accuracy": report["accuracy"], # 使用前面计算的报告中的准确率

    # Per-class metrics
    "precision_0": report['0']['precision'],
    "recall_0": report['0']['recall'],
    "f1_0": report['0']['f1-score'],
    "precision_1": report['1']['precision'],
    "recall_1": report['1']['recall'],
    "f1_1": report['1']['f1-score'],

    # Micro-average (使用手动计算的值)
    "precision_micro": precision_score(all_true_labels, final_preds_labels, average='micro', zero_division=0),
    "recall_micro": recall_score(all_true_labels, final_preds_labels, average='micro', zero_division=0),
    "f1_micro": f1_score(all_true_labels, final_preds_labels, average='micro', zero_division=0),

    # Macro-average
    "precision_macro": report['macro avg']['precision'],
    "recall_macro": report['macro avg']['recall'],
    "f1_macro": report['macro avg']['f1-score'],

    # Weighted Averages
    "precision_weighted": report['weighted avg']['precision'],
    "recall_weighted": report['weighted avg']['recall'],
    "f1_weighted": report['weighted avg']['f1-score'],

    "lr_max": training_args.learning_rate,
    "lr_min": 0.0,
    "best_lr": best_lr, # 使用上面更新的 best_lr
    "epoch_stopped_at": int(epoch_stopped_at), # 使用上面更新的 epoch_stopped_at

    # Class balance info
    "support_1": report['1']['support'],
    "support_0": report['0']['support'],
    "train_time_sec": train_time # 使用前面计算的 train_time
}

# Excel Integration
df_row = pd.DataFrame([summary_row])
if not summary_path.exists():
    df_row.to_excel(summary_path, index=False)
else:
    existing = pd.read_excel(summary_path)
    updated = pd.concat([existing, df_row], ignore_index=True)
    updated.to_excel(summary_path, index=False)

print(f"✅ 模型评估总表已更新并保存至：{summary_path}")

✅ 模型评估总表已更新并保存至：/kaggle/working/MSc_Claim_Experiment/outputs/results/final_model_comparison.xlsx


In [ ]:
# 提前构造路径
model_path = project_root / "outputs" / "results" / "saved_models" / f"{model_name.replace('/', '_')}_{run_id}"

# 打印输出
print(f"✅ 最佳模型权重已保存至: {model_path}")

In [31]:
import shutil

# Backup all output files (including intermediate files, models, logs, etc.)
# This creates a full archive of the experiment outputs
shutil.make_archive(
    "/kaggle/working/results_backup", # 打包后的zip文件路径和名称
    'zip',                             # 压缩格式
    "/kaggle/working/MSc_Claim_Experiment/outputs" # 要打包的源目录
)

'/kaggle/working/results_backup.zip'

In [ ]:
#-------------------------------------------------------------------------------------------

In [ ]:
from transformers import AutoModelForSequenceClassification


# ===== Predict and Save Outputs ===== #
# Generate predictions from best model on the test set
preds = trainer.predict(test_ds)

In [ ]:
# Save the best model weights for reproducibility or future use
trainer.save_model(project_root / "outputs" / "results" / "saved_models" / f"{model_name.replace('/', '_')}_{run_id}")

In [ ]:
# Extract predicted labels
preds_labels = np.argmax(preds.predictions, axis=-1)

In [ ]:
# ===== Save Raw Prediction Table ===== #
# Save full test set results with input, true label, predicted label, and logits
results = pd.DataFrame({
    'statement': test['statement'].reset_index(drop=True),
    'true_label': test['label_binary'].reset_index(drop=True),
    'pred_label': preds_labels,
    'logits': preds.predictions.tolist(),
    'model_name': model_name,
    'run_id': run_id
})
results.to_csv(output_dir / "prediction.csv", index=False)

In [ ]:
# ===== Save Metrics =====
# Save raw evaluation metrics to JSON
with open(output_dir / "eval_metrics.json", "w") as f:
    json.dump(preds.metrics, f, indent=2)

In [ ]:
# ===== Save Report & Confusion Matrix =====
# Save detailed classification report and confusion matrix
conf_matrix = confusion_matrix(results['true_label'], results['pred_label'])
report = classification_report(results['true_label'], results['pred_label'], output_dict=True)

with open(output_dir / "classification_report.json", "w") as f:
    json.dump(report, f, indent=2)

with open(output_dir / "confusion_matrix.json", "w") as f:
    json.dump(conf_matrix.tolist(), f, indent=2)


In [ ]:
# === Estimate learning rate & early stop epoch ===
best_lr = trainer.optimizer.param_groups[0]['lr']
steps_per_epoch = len(train_ds) // training_args.per_device_train_batch_size
epoch_stopped_at = trainer.state.global_step // steps_per_epoch

print(f"Final learning rate (from best checkpoint): {best_lr}")
print(f"Early stopped at epoch: {epoch_stopped_at}")

# save best_lr to file
with open(output_dir / "best_lr.txt", "w") as f:
    f.write(str(best_lr))

In [ ]:
# Save model & training configuration before training
config_snapshot = {
    "model_name": model_name,
    "run_id": run_id,

     # Hyperparameters
    "learning_rate": training_args.learning_rate,
    "lr_scheduler_type": training_args.lr_scheduler_type,
    "warmup_ratio": training_args.warmup_ratio,
    "batch_size": training_args.per_device_train_batch_size,
    "num_epochs": training_args.num_train_epochs,
    "weight_decay": training_args.weight_decay,
    "early_stopping": True,

    "best_lr": best_lr,
    "epoch_stopped_at": epoch_stopped_at,
    
    "class_weights": class_weights.cpu().tolist(),
     "datetime": timestamp
    #"datetime": datetime.now().isoformat()
}
with open(output_dir / "config.json", "w") as f:
    json.dump(config_snapshot, f, indent=2)

# Write human-readable run info for easy reference
with open(output_dir / "README.txt", "w") as f:
    f.write(f"Run ID: {run_id}\nModel: {model_name}\nCreated: {timestamp}\n")
    f.write(f"Best LR (from checkpoint): {best_lr}\nStopped at epoch: {epoch_stopped_at}\n")

In [ ]:
# ===== Save TrainingArguments Params (for reproducibility) ===== #
with open(output_dir / "training_args.json", "w") as f:
    json.dump(training_args.to_dict(), f, indent=2)

# ===== Print summary ===== #
print(f"All results saved to: {output_dir}")



# ===== Append Main Metrics to Excel Summary Table ===== #
# Create/update a summary Excel for final model comparisons
summary_path = project_root / "outputs" / "results" / "final_model_comparison.xlsx"

# Re-run prediction to ensure use final model output
preds = trainer.predict(test_ds)
y_true = preds.label_ids
y_pred = preds.predictions.argmax(-1)

# Compute detailed classification report
report = classification_report(
    y_true, y_pred,
    output_dict=True,
    zero_division=0
)

# Manually compute micro metrics to avoid missing values
precision_micro = precision_score(y_true, y_pred, average='micro', zero_division=0)
recall_micro = recall_score(y_true, y_pred, average='micro', zero_division=0)
f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0)

# One row summarizing performance of this model run
summary_row = {
    "model_name": model_name,
    "run_id": run_id,
    "accuracy": preds.metrics.get("test_accuracy", preds.metrics.get("eval_accuracy", report["accuracy"])),
 
    # Per-class metrics
    "precision_0": report['0']['precision'],
    "recall_0": report['0']['recall'],
    "f1_0": report['0']['f1-score'],
    "precision_1": report['1']['precision'],
    "recall_1": report['1']['recall'],
    "f1_1": report['1']['f1-score'],

    # Micro-average
    "precision_micro": precision_micro,
    "recall_micro": recall_micro,
    "f1_micro": f1_micro,

    # Macro-average
    "precision_macro": report['macro avg']['precision'],
    "recall_macro": report['macro avg']['recall'],
    "f1_macro": report['macro avg']['f1-score'],

    # Weighted Averages
    "precision_weighted": report['weighted avg']['precision'], 
    "recall_weighted": report['weighted avg']['recall'], 
    "f1_weighted": report['weighted avg']['f1-score'],

    "lr_max": training_args.learning_rate,
    "lr_min": 0.0,  
    "best_lr": best_lr,
    "epoch_stopped_at": epoch_stopped_at,

     # Class balance info
    "support_1": report['1']['support'],
    "support_0": report['0']['support'],
    "train_time_sec": train_time

}

# ===== Excel Integration ===== #
# Append or create Excel summary table for all runs

df_row = pd.DataFrame([summary_row])
if not summary_path.exists():
    df_row.to_excel(summary_path, index=False)
else:
    existing = pd.read_excel(summary_path)
    updated = pd.concat([existing, df_row], ignore_index=True)
    updated.to_excel(summary_path, index=False)

print(f"Summary row saved to {summary_path}")

In [ ]:
import os

base_path = "/kaggle/working/MSc_Claim_Experiment/outputs/results"

for root, dirs, files in os.walk(base_path):
    for file in files:
        print(os.path.join(root, file))

In [ ]:
import pandas as pd
from pathlib import Path

# === 1. Define Base Path Depending on Platform ===
if Path("/kaggle/working").exists():
    base_path = Path("/kaggle/working/MSc_Claim_Experiment/outputs/results")
elif Path("/content/drive").exists():
    base_path = Path("/content/drive/MyDrive/MSc_Claim_Experiment/outputs/results")
else:
    base_path = Path("./outputs/results")  # fallback for local dev

# === 2. Load model summary file ===
df_summary = pd.read_excel(base_path / "final_model_comparison.xlsx")
print("Final Model Comparison:")
display(df_summary.head())

# === 3. Define model_keyword (before calling the function) ===
model_keyword = "None"  # Change to "bert", "distilbert", or None

# === 4. Define reusable function ===
def get_latest_run_folder(results_path: Path, model_keyword: str = None) -> Path:
    """
    Get the most recently updated results folder.
    If model_keyword is provided, only consider folders containing that keyword.
    """
    subdirs = [d for d in results_path.iterdir() if d.is_dir()]
    if model_keyword:
        subdirs = [d for d in subdirs if model_keyword.lower() in d.name.lower()]
    return max(subdirs, key=lambda d: d.stat().st_mtime) if subdirs else None

# === 5. Get latest result folder and load predictions ===
latest_run_folder = get_latest_run_folder(base_path, model_keyword=model_keyword)

if latest_run_folder:
    df_pred = pd.read_csv(latest_run_folder / "prediction.csv")
    print(f"Sample Predictions from: {latest_run_folder.name}")
    display(df_pred.head())
else:
    print("No valid result folder found.")

In [ ]:
# import shutil

# Backup all output files (including intermediate files, models, logs, etc.)
# This creates a full archive of the experiment outputs
shutil.make_archive(
    "/kaggle/working/results_backup", 
    'zip', 
    "/kaggle/working/MSc_Claim_Experiment/outputs"
)